In [1]:
#Import libraries
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate


Read input files

In [2]:
#Read input files
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')

In [3]:
n_train=train.copy()

#Extract title
n_train['Title']=n_train['Name'].apply(lambda x: x.split('.')[0].strip())
n_train['Title']=n_train['Title'].apply(lambda x: x.split(',')[-1].strip())

#Extract Deck
n_train['Deck']=n_train['Cabin'].str.extract(r'([a-zA-Z]*)')
n_train['Deck'].fillna('Z', inplace=True)
n_train['Deck']=n_train['Deck'].where(n_train['Deck']!='T', 'G')

#Extract Cabin number
n_train['Cab_No']=n_train['Cabin'].str.extract(r'(\d+)')
n_train['Cab_No'].fillna(0, inplace=True)

In [4]:
#n_train.to_csv("n_train.csv",header=True,index=False)

Modify Test and Train files to simplify the Titles - as too many categories give the program too little data to train for each category

We end up with 2 titles for men and 1 for women - the reason is that most women survived and most men died so there is no reason to distinguish between women but some men had better changes than others so adding a second category makes sense.

I've also created a category for children but it didn't do much good. Survival of children wasn't great.

Filling missing values - for missing age I inserted 999 for missing fare I inserted 0.01 and for missing cabin number I inserted 0. 
It may be better to use average values for age e.g. based on Sex, Class and embarcation harbour but I didn't test it. 

In [5]:
n_train=train.copy()
n_train['Title']=n_train['Name'].apply(lambda x: x.split('.')[0].strip())
n_train['Title']=n_train['Title'].apply(lambda x: x.split(',')[-1].strip())
n_train['Title'] = n_train['Title'].replace('Capt', 'Mr', regex=True)
n_train['Title'] = n_train['Title'].replace('Col', 'Master', regex=True)
n_train['Title'] = n_train['Title'].replace('Major', 'Master', regex=True)
n_train['Title'] = n_train['Title'].replace('Dr', 'Master', regex=True)
n_train['Title'] = n_train['Title'].replace('Don', 'Mr', regex=True)
n_train['Title'] = n_train['Title'].replace('Jonkheer', 'Mr', regex=True)
n_train['Title'] = n_train['Title'].replace('Rev', 'Mr', regex=True)
n_train['Title'] = n_train['Title'].replace('Sir', 'Master', regex=True)
n_train['Title'] = n_train['Title'].replace('Lady', 'Mrs', regex=True)
n_train['Title'] = n_train['Title'].replace('Mme', 'Mrs', regex=True)
n_train['Title'] = n_train['Title'].replace('the Countess', 'Mrs', regex=True)
n_train['Title'] = n_train['Title'].replace('Mlle', 'Mrs', regex=True)
n_train['Title'] = n_train['Title'].replace('Miss', 'Mrs', regex=True)
n_train['Title'] = n_train['Title'].replace('Ms', 'Mrs', regex=True)
n_train['Embarked'].fillna("S", inplace = True)
n_train['Age'].fillna(999, inplace = True)
n_train['Fare'].fillna(0.01, inplace = True)
n_train['Child']=(n_train['Age']<18).astype(int)

n_train['Deck']=n_train['Cabin'].str.extract(r'([a-zA-Z]*)')
n_train['Deck'].fillna('Z', inplace=True)
n_train['Deck']=n_train['Deck'].where(n_train['Deck']!='T', 'Z')

n_train['Cab_No']=n_train['Cabin'].str.extract(r'(\d+)')
n_train['Cab_No'].fillna(0, inplace=True)
n_train['Cab_No']=n_train['Cab_No'].where(n_train['Cab_No']==0, 1)


n_train= n_train[["Pclass", "Sex", "Embarked", "Title", "Child", "Fare", "Deck", "Cab_No", "Survived"]]
x_train= n_train[["Pclass", "Sex", "Embarked", "Title", "Child", "Fare", "Deck", "Cab_No"]]
#x_train= n_train[["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Length"]]
y_train= n_train[["Survived"]]

n_test=test.copy()
n_test['Title']=n_test['Name'].apply(lambda x: x.split('.')[0].strip())
n_test['Title']=n_test['Title'].apply(lambda x: x.split(',')[-1].strip())
n_test['Title'] = n_test['Title'].replace('Dona', 'Mrs', regex=True)
n_test['Title'] = n_test['Title'].replace('Ms', 'Mrs', regex=True)
n_test['Title'] = n_test['Title'].replace('Rev', 'Mr', regex=True)
n_test['Title'] = n_test['Title'].replace('Col', 'Master', regex=True)
n_test['Title'] = n_test['Title'].replace('Miss', 'Mrs', regex=True)
n_test['Title'] = n_test['Title'].replace('Dr', 'Master', regex=True)

n_test['Deck']=n_test['Cabin'].str.extract(r'([a-zA-Z]*)')
n_test['Deck'].fillna('Z', inplace=True)

n_test['Cab_No']=n_test['Cabin'].str.extract(r'(\d+)')
n_test['Cab_No'].fillna(0, inplace=True)
n_test['Cab_No']=n_test['Cab_No'].where(n_test['Cab_No']==0, 1)

n_test['Age'].fillna(999, inplace = True)
n_test['Child']=(n_test['Age']<18).astype(int)
n_test['Fare'].fillna(0.01, inplace = True)
#n_test.to_csv("n_test1.csv",header=True,index=False)
x_test=n_test[["Pclass", "Sex", "Embarked", "Title", "Child", "Fare", "Deck", "Cab_No"]]
n_train.to_csv("n_train2.csv",header=True,index=False)

Encode Categorical columns

In [6]:
categorical_cols=['Sex', 'Embarked', 'Title', 'Deck']
x_train_t=pd.get_dummies(x_train, columns=categorical_cols)
x_test_t=pd.get_dummies(x_test, columns=categorical_cols)

Convert True and False to 1 and 0

In [7]:
x_train_t=x_train_t*1
x_test_t=x_test_t*1

In [8]:
x_train_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Pclass        891 non-null    int64  
 1   Child         891 non-null    int64  
 2   Fare          891 non-null    float64
 3   Cab_No        891 non-null    object 
 4   Sex_female    891 non-null    int64  
 5   Sex_male      891 non-null    int64  
 6   Embarked_C    891 non-null    int64  
 7   Embarked_Q    891 non-null    int64  
 8   Embarked_S    891 non-null    int64  
 9   Title_Master  891 non-null    int64  
 10  Title_Mr      891 non-null    int64  
 11  Title_Mrs     891 non-null    int64  
 12  Deck_A        891 non-null    int64  
 13  Deck_B        891 non-null    int64  
 14  Deck_C        891 non-null    int64  
 15  Deck_D        891 non-null    int64  
 16  Deck_E        891 non-null    int64  
 17  Deck_F        891 non-null    int64  
 18  Deck_G        891 non-null    

Change data to Values

In [9]:
#x_test_t.info()

In [10]:
x_train_2=x_train_t.iloc[:, :].values
x_test_2=x_test_t.iloc[:, :].values

Scale the features in the non categorical columns

In [11]:
sc = StandardScaler()
x_train_2[:, :4] = sc.fit_transform(x_train_2[:, :4])
x_test_2[:, :4] = sc.transform(x_test_2[:, :4])
y_train_2=y_train.values.ravel()

Convert every array to Float 32

In [12]:
x_train_2 = np.asarray(x_train_2).astype(np.float32)
x_test_2 = np.asarray(x_test_2).astype(np.float32)

Here I start to apply different models to get my predicted survival numbers.

Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state = 0, max_iter = 1000)
log_reg.fit(x_train_2, y_train_2)

LogisticRegression(max_iter=1000, random_state=0)

In [14]:
y_pred = log_reg.predict(x_test_2)
y_pred_log= (np.concatenate(y_pred.reshape(len(y_pred),1,1)))

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = y_pred_log
submission.to_csv("submission_Log_Reg.csv",header=True,index=False)

K-Nearest Neighbours

In [15]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=9)
KNN.fit(x_train_2, y_train_2)

KNeighborsClassifier(n_neighbors=9)

In [16]:
knn_pred=KNN.predict(x_test_2)
knn_pred_f=(np.concatenate(knn_pred.reshape(len(knn_pred),1,1)))

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = knn_pred_f
submission.to_csv("submission_KNN_9.csv",header=True,index=False)

Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train_2, y_train_2)

DecisionTreeClassifier()

In [18]:
decision_tree_pred=decision_tree.predict(x_test_2)
decision_tree_pred_f=(np.concatenate(decision_tree_pred.reshape(len(decision_tree_pred),1,1)))

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = decision_tree_pred_f
submission.to_csv("submission_Dec_Tree.csv",header=True,index=False)

Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
random_forest.fit(x_train_2, y_train_2)

random_forest_pred=random_forest.predict(x_test_2)
random_forest_pred_f=(np.concatenate(random_forest_pred.reshape(len(random_forest_pred),1,1)))

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = random_forest_pred_f
submission.to_csv("submission_Rand_Forest.csv",header=True,index=False)



Support Vector Machine

In [20]:
from sklearn.svm import SVC
svm_model = SVC(probability=True, kernel="rbf" ,degree=5 ,random_state=42)
svm_model.fit(x_train_2, y_train_2)

svm_model_pred=svm_model.predict(x_test_2)
svm_model_pred_f=(np.concatenate(svm_model_pred.reshape(len(svm_model_pred),1,1)))

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = svm_model_pred_f
submission.to_csv("submission_SVM_rbf_5.csv",header=True,index=False)

Gradient Boosting

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
gradient_boosting = GradientBoostingClassifier()
gradient_boosting.fit(x_train_2, y_train_2)

gradient_boosting_pred=gradient_boosting.predict(x_test_2)
gradient_boosting_pred_f=(np.concatenate(gradient_boosting_pred.reshape(len(gradient_boosting_pred),1,1)))

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = gradient_boosting_pred_f
submission.to_csv("submission_Gradient_Boost.csv",header=True,index=False)

MLP Classifier

In [22]:
from sklearn.neural_network import MLPClassifier
mlp_classifier=MLPClassifier(max_iter=5000).fit(x_train_2, y_train_2)
mlp_classifier_pred=mlp_classifier.predict(x_test_2)

In [23]:
#Predict Survivor numbers
nn_pred_f=(np.concatenate(mlp_classifier_pred.reshape(len(mlp_classifier_pred),1,1)))

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = nn_pred_f
submission.to_csv("submission_nn3.csv",header=True,index=False)

Below we take 3 of the above results and weigh them.
It is possible to uncomment the other models e.g. Logistic Regression - when doing so it is necessary to add another position in the weights list to keep the nubmer of models used the same as the number of numbers in the list.

The current weightings "weights=[1, 2, 1]" give SVM twice the weight of KNN of ANN.

In [24]:
# Voting prediction
from sklearn.ensemble import VotingClassifier

VOTING = VotingClassifier([
#    ("Logistic Regression", log_reg),
    ("KNN", KNN),
#    ("Decision Tree", decision_tree),
#    ("Random Forest", random_forest),
    ("SVM", svm_model),
#    ("Gradient Boosting", gradient_boosting),
    ("ANN", mlp_classifier)
], voting="soft", weights=[1, 2, 1])

VOTING.fit(x_train_2, y_train.values.ravel())

test_prediction = VOTING.predict(x_test_2)

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = test_prediction
submission.to_csv("Voting_S_1.csv",header=True,index=False)

Stacking Prediction

This is another way of weighing the results of the models to obtain a better result.

In [25]:
from sklearn.ensemble import StackingClassifier
STACK = StackingClassifier([
#    ("Logistic Regression", log_reg),
    ("KNN", KNN),
#    ("Decision Tree", decision_tree),
#    ("Random Forest", random_forest),
    ("SVM", svm_model),
#   ("Gradient Boosting", gradient_boosting),
    ("ANN", mlp_classifier)])
STACK.fit(x_train_2, y_train_2)
test_stack_prediction = STACK.predict(x_test_2)

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = test_stack_prediction
submission.to_csv("Stacking_S_1.csv",header=True,index=False)

Ensemble all results

In [26]:
ensemble=((test_stack_prediction*0.3) + (test_prediction*0.7))
ensemble_round = np.where((ensemble > 0.5), 1, 0)

submission = pd.DataFrame()
submission["PassengerId"] = test["PassengerId"]
submission["Survived"] = ensemble_round
submission.to_csv("Ensemble_max_1.csv",header=True,index=False)